In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Input, Flatten, MaxPooling2D
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import cv2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## Defining class to support Tree-CNN

In [46]:
class CNN (object):
    def __init__(self, num_classes, input_shape):
#         input_layer, l = self.create_network_base(num_classes, input_shape)
        self.num_classes = num_classes
        self.input_shape = input_shape
        self.model = tf.keras.applications.resnet.ResNet50(include_top=True, weights=None, classes=num_classes, input_shape=input_shape)
#         Model(input_layer, l)
        self.model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    
#     def create_network_base(self, num_classes, input_shape):
#         input_layer = Input(shape=input_shape)
#         l = Conv2D(64, 3, activation='relu')(input_layer)
#         l = MaxPooling2D()(l)
#         l = Conv2D(32, 3, activation='relu')(l)
#         l = MaxPooling2D()(l)
#         l = Conv2D(32, 3,activation='relu')(l)
#         l = MaxPooling2D()(l)
#         l = Conv2D(16, 3, activation='relu')(l)
#         l = MaxPooling2D()(l)
#         l = Flatten()(l)
#         l = Dense(64, activation='relu')(l)
#         l = Dense(num_classes, activation='softmax')(l)
        
#         return input_layer, l
    
    def train(self, X, Y):
        cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
        print(X.shape, Y.shape)
        self.model.fit(X, Y, batch_size=16, epochs=100000, callbacks=[cb], validation_split=0.1)

    def remove_class(self, idx_to_remove):
#         input_layer, l = self.create_network_base((self.num_classes-1), self.input_shape)
        new_model = tf.keras.applications.resnet.ResNet50(include_top=True, weights=None, classes=(self.num_classes-1), input_shape=self.input_shape)
#     Model(input_layer, l)
        
        for idx in range(len(self.model.layers)-1):
            if len(self.model.layers[idx].get_weights()) == 0 :
                continue
            if len(self.model.layers[idx].get_weights()) == 2 :
                wi = self.model.layers[idx].get_weights()[0]
                bi = self.model.layers[idx].get_weights()[1]
                new_model.layers[idx].set_weights((wi, bi))
            if len(self.model.layers[idx].get_weights()) == 4 :
                wi = self.model.layers[idx].get_weights()[0]
                bi = self.model.layers[idx].get_weights()[1]
                wi2 = self.model.layers[idx].get_weights()[2]
                bi2 = self.model.layers[idx].get_weights()[3]
                new_model.layers[idx].set_weights((wi, bi, wi2, bi2))
        
        # Copy a already treined part of last layer
        old_w = self.model.layers[-1].get_weights()[0]
        new_w = new_model.layers[-1].get_weights()[0]
        old_bias = self.model.layers[-1].get_weights()[1]
        new_bias = new_model.layers[-1].get_weights()[1]

        for i in range(old_w.shape[0]):
            aux = 0
            for j in range(old_w.shape[1]):
                if j != idx_to_remove:
                    new_w[i][aux] = old_w[i][j]
                    aux = aux + 1
        aux = 0
        for i in range(old_bias.shape[0]):
            if i != idx_to_remove:
                new_bias[aux] = old_bias[i]
                aux = aux + 1

        new_model.layers[-1].set_weights((new_w, new_bias))
        
        self.model = new_model
        self.model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
        self.num_classes = self.num_classes - 1
        
    def add_class(self):
#         input_layer, l = self.create_network_base((self.num_classes+1), self.input_shape)
        new_model = tf.keras.applications.resnet.ResNet50(include_top=True, weights=None, classes=(self.num_classes+1), input_shape=self.input_shape)
#     Model(input_layer, l)
        
        for idx in range(len(self.model.layers)-1):
            if len(self.model.layers[idx].get_weights()) == 0 :
                continue
            if len(self.model.layers[idx].get_weights()) == 2 :
                wi = self.model.layers[idx].get_weights()[0]
                bi = self.model.layers[idx].get_weights()[1]
                new_model.layers[idx].set_weights((wi, bi))
            if len(self.model.layers[idx].get_weights()) == 4 :
                wi = self.model.layers[idx].get_weights()[0]
                bi = self.model.layers[idx].get_weights()[1]
                wi2 = self.model.layers[idx].get_weights()[2]
                bi2 = self.model.layers[idx].get_weights()[3]
                new_model.layers[idx].set_weights((wi, bi, wi2, bi2))
        
        # Copy a already treined part of last layer
        old_w = self.model.layers[-1].get_weights()[0]
        new_w = new_model.layers[-1].get_weights()[0]
        old_bias = self.model.layers[-1].get_weights()[1]
        new_bias = new_model.layers[-1].get_weights()[1]

        for i in range(old_w.shape[0]):
            for j in range(old_w.shape[1]):
                new_w[i][j] = old_w[i][j]
        for i in range(old_bias.shape[0]):
            new_bias[i] = old_bias[i]

        new_model.layers[-1].set_weights((new_w, new_bias))
        
        self.model = new_model
        self.model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
        self.num_classes = self.num_classes + 1
        
    def pred(self, img):
        return self.model.predict(img)

In [47]:
class NodeList (object):
    def __init__(self, label, values, nodes, class_position):
        self.label = label
        self.values = values
        self.nodes = nodes
        self.class_position = class_position
    def __str__(self):
        return {'label': self.label, 'values': self.values, 'nodes':self.nodes, 'class_position': self.class_position}
    def __unicode__(self):
        return {'label': self.label, 'values': self.values, 'nodes':self.nodes, 'class_position': self.class_position}
    def __repr__(self):
        return 'label: ' + str(self.label) + ' values: ' + str(self.values) + ' nodes: ' + str(self.nodes)+' class_pos: '+ str(self.class_position)

In [48]:
class CnnNode (object):
    def __init__(self, num_classes, labels = [], input_shape=(28,28,1), max_leafes=10):
        self.net = CNN(num_classes, input_shape)
        self.num_classes = num_classes
        self.childrens = [label for label in labels]
        self.childrens_leaf = [True for _ in range(num_classes)]
        self.labels = labels
        self.max_leafes = max_leafes
        self.labels_transform = {}
        for nc in range(num_classes):
            self.labels_transform[labels[nc]] = []
            self.labels_transform[labels[nc]].append(labels[nc])
        
    
    def get_num_leafnodes(self):
        count = 0
        for is_leaf in self.childrens_leaf:
            if is_leaf:
                count = count + 1
        return count
    
    def remove_leaf(self, label):
        childrens = []
        childrens_leaf = []
        labels = []
        del self.labels_transform[label]
        self.num_classes = (self.num_classes - 1)
        position_in_net = -1
        
        for i in range(len(self.labels)):
            if self.labels[i] != label:
                childrens.append(self.childrens[i])
                childrens_leaf.append(self.childrens_leaf[i])
                labels.append(self.labels[i])
            else:
                position_in_net = i
                
        self.childrens = childrens
        self.childrens_leaf = childrens_leaf
        self.labels = labels
        self.net.remove_class(position_in_net)
        
    def add_leaf(self, label):
        self.childrens.append(label)
        self.childrens_leaf.append(True)
        self.num_classes = (self.num_classes + 1)
        self.labels.append(label)
        self.labels_transform[label] = []
        self.labels_transform[label].append(label)
        self.net.add_class()
    
    def predict(self, imgs):
        vector_output = self.net.pred(imgs)
        return vector_output
    
    def inference(self, imgs):
        vector_output = self.predict(imgs)
        out = np.array([idx for idx in np.argmax(vector_output, axis=1)])
        output = np.array([-1 for _ in range(imgs.shape[0])])

        for i, o in zip(range(len(out)), out):
            if self.childrens_leaf[o]:
                output[i] = self.labels[o]            

        # Send images to branches nodes
        for child_id in range(len(self.childrens_leaf)):
            if not self.childrens_leaf[child_id]:
                output[out==child_id] = self.childrens[child_id].inference(imgs[out==child_id])


        return output
    
    def train(self, X, Y):
        Y_to_this_nivel = Y.copy()
        print(1, self.labels_transform)
        for idx in range(len(self.labels)):
            for label in self.labels_transform[self.labels[idx]]:
                Y_to_this_nivel[Y==label] = idx

        Y_true = pd.get_dummies(pd.Series(Y_to_this_nivel)).values
        self.net.train(X, Y_true)
        
        # Send images to branches nodes
        for child_id in range(len(self.childrens_leaf)):
            if not self.childrens_leaf[child_id]:
                self.childrens[child_id].train(X[Y_to_this_nivel==child_id], Y[Y_to_this_nivel==child_id])

In [49]:
class TreeCNN (object):
    def __init__(self, initial_labels, alpha=0.1, beta=0.1, input_shape=(28,28,1), max_leafnodes=1000):
        num_class_initial = len(initial_labels)
        self.root = CnnNode(num_class_initial, labels=initial_labels, input_shape=input_shape)
        self.alpha = alpha
        self.beta = beta
        self.max_leafnodes = max_leafnodes
        
    
    def addTasks(self, imgs_of_classes=[], labels=[]):
        self.growTreeCNN(self.root, imgs_of_classes, labels)
        
    def train(self, X, Y):
        self.root.train(X, Y)
        
    def inference(self, X):
        return self.root.inference(X)
        
        
    def growTreeCNN(self, operation_node, imgs_of_classes=[], labels=[]):
        def get_Oavg_matrix(node, imgs_of_classes_, labels_):
            Oavg = np.zeros(shape=(node.num_classes, 0))
            for imgs, label in zip(imgs_of_classes_, labels_):
                net_out = node.predict(imgs)
                Oavg_i = np.average(net_out, axis=0)
                Oavg = np.concatenate(( Oavg, Oavg_i.reshape((Oavg_i.shape[0], 1)) ), axis=1)
            return Oavg
        
        def get_loglikelihood_matrix(Oavg):
            return (np.power(np.e, Oavg) / np.sum(np.power(np.e, Oavg), axis=0))
        
        def generate_listS(llh, labels_in):
            listS = []
            for i in range(llh.shape[1]):
                label = labels_in[i]
                values = []
                nodes = []

                col = llh[:,i].copy()
                for _ in range(3):
                    max_idx = np.argmax(col)
                    values.append(col[max_idx])
                    nodes.append(max_idx)
                    col[max_idx] = -100

                listS.append(NodeList(label, values, nodes, i))

            # Sort List S by value of S[i].values[0]
            listS.sort(key=lambda node_list: node_list.values[0])
            
            return listS

        llh = get_loglikelihood_matrix(get_Oavg_matrix(operation_node, imgs_of_classes, labels))
        
        listS = generate_listS(llh, labels)
        new_labels = labels
        
        branches_dest = {}
        while len(listS) > 0:
            nodeList = listS[0]
            rows_to_remove_in_llh = []
            if nodeList.values[0] - nodeList.values[1] > self.alpha:
                if operation_node.childrens_leaf[nodeList.nodes[0]]:
                    operation_node.childrens_leaf[nodeList.nodes[0]] = False
                    old_label = operation_node.labels[nodeList.nodes[0]]
                    new_label = nodeList.label
                    branch_node = CnnNode(2, labels=[old_label, new_label])
                    operation_node.childrens[nodeList.nodes[0]] = branch_node
                else:
                    if nodeList.nodes[0] not in branches_dest:
                        branches_dest[nodeList.nodes[0]] = []
                    branches_dest[nodeList.nodes[0]].append(nodeList.label)
                
                operation_node.labels_transform[nodeList.nodes[0]].append(nodeList.label)
            elif nodeList.values[1] - nodeList.values[2] > self.beta:
                left_is_leafnode = operation_node.childrens_leaf[nodeList.nodes[0]]
                right_is_leafnode = operation_node.childrens_leaf[nodeList.nodes[1]]
                has_space_in_left = left_is_leafnode or (operation_node.childrens[nodeList.nodes[0]].get_num_leafnodes() < (self.max_leafnodes - 1))
                
                if right_is_leafnode and has_space_in_left: # if Merge
                    if operation_node.childrens_leaf[nodeList.nodes[0]]: # if left is a leaf
                        operation_node.childrens_leaf[nodeList.nodes[0]] = False
                        old_label = operation_node.labels[nodeList.nodes[0]]
                        new_label = operation_node.labels[nodeList.nodes[1]]
                        branch_node = CnnNode(2, labels=[old_label, new_label])
                        operation_node.childrens[nodeList.nodes[0]] = branch_node
                    else:
                        operation_node.childrens[nodeList.nodes[0]].add_leaf(operation_node.labels[nodeList.nodes[1]])
                        if nodeList.nodes[0] not in branches_dest:
                            branches_dest[nodeList.nodes[0]] = []
                        branches_dest[nodeList.nodes[0]].append(nodeList.label)
                    
                    operation_node.labels_transform[nodeList.nodes[0]].append(operation_node.labels[nodeList.nodes[1]])
                    operation_node.remove_leaf(operation_node.labels[nodeList.nodes[1]])
                    rows_to_remove_in_llh.append(nodeList.nodes[1])
                else:
                    if left_is_leafnode:
                        operation_node.childrens_leaf[nodeList.nodes[0]] = False
                        old_label = operation_node.labels[nodeList.nodes[0]]
                        new_label = nodeList.label
                        branch_node = CnnNode(2, labels=[old_label, new_label])
                        operation_node.childrens[nodeList.nodes[0]] = branch_node
                    elif right_is_leafnode:
                        operation_node.childrens_leaf[nodeList.nodes[1]] = False
                        old_label = operation_node.labels[nodeList.nodes[1]]
                        new_label = nodeList.label
                        branch_node = CnnNode(2, labels=[old_label, new_label])
                        operation_node.childrens[nodeList.nodes[1]] = branch_node
                    else:
                        if operation_node.childrens[nodeList.nodes[0]].get_num_leafnodes() < operation_node.childrens[nodeList.nodes[1]].get_num_leafnodes():
                            if nodeList.nodes[0] not in branches_dest:
                                branches_dest[nodeList.nodes[0]] = []
                            branches_dest[nodeList.nodes[0]].append(nodeList.label)
                        else:
                            if nodeList.nodes[1] not in branches_dest:
                                branches_dest[nodeList.nodes[1]] = []
                                print(nodeList)
                            branches_dest[nodeList.nodes[1]].append(nodeList.label)
                    
            else:
                operation_node.add_leaf(nodeList.label)
                
            
            # Clean likelihood matrix and labels to recreate listS to next iteration
            # Delete column already inserted
            llh = np.delete(llh, nodeList.class_position, axis=1)
            # Delete rows that was merged
            for r in rows_to_remove_in_llh:
                llh = np.delete(llh, r, axis=0)
            # Delete rows that represent full nodes (just branch/child nodes)
            deleted = 0
            for i in range(len(operation_node.childrens_leaf)):
                if not operation_node.childrens_leaf[i] and (operation_node.childrens[i].get_num_leafnodes() >= self.max_leafnodes):
                    llh = np.delete(llh, (i - deleted), axis=0)
                    deleted = deleted + 1
            # Update labels
            del new_labels[nodeList.class_position]
            listS = generate_listS(llh, new_labels)
        
        # Send to sub-nivels
        for k, v in branches_dest.items():
            imgs_to_send = []
            labels_to_send = []
            for idx, label in zip(range(len(labels)), labels):
                if label in v:
                    imgs_to_send.append(imgs_of_classes[idx])
                    labels_to_send.append(labels[idx])

            self.growTreeCNN(operation_node.childrens[k], imgs_to_send, labels_to_send)
        

In [ ]:
tree = TreeCNN([0,1,2,3])

## Test with MNIST

In [ ]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
num_initial_classes = 4
old_classes_X = x_train[y_train<num_initial_classes].astype(float)
old_classes_X = old_classes_X.reshape((old_classes_X.shape[0], old_classes_X.shape[1], old_classes_X.shape[2], 1))
old_classes_Y = y_train[y_train<num_initial_classes]

new_classes_X = [x_train[y_train==i].reshape((x_train[y_train==i].shape[0], x_train[y_train==i].shape[1], x_train[y_train==i].shape[2], 1 )).astype(float) for i in range(4, 7)]
new_classes_Y = [i for i in range(4, 7)]


# pd.get_dummies(pd.Series(old_classes_Y))
merged_classes_X = x_train[y_train<7].astype(float)
merged_classes_X = merged_classes_X.reshape((merged_classes_X.shape[0], merged_classes_X.shape[1], merged_classes_X.shape[2], 1))
merged_classes_Y = y_train[y_train<7]

### Train with firsts classes

In [ ]:
np.unique(new_classes_Y)

In [ ]:
tree.train(old_classes_X, old_classes_Y)

In [ ]:
Y_hat = tree.inference(old_classes_X)
Y_true = y_train[y_train<num_initial_classes]
print(Y_hat)
np.sum(Y_true==Y_hat)/len(Y_true)

### Growing network

In [ ]:
tree.addTasks(new_classes_X, [i for i in range(4, 7)])

In [ ]:
print(tree.root.net.model.layers[-1].output_shape)
print(tree.root.childrens_leaf)
print(tree.root.childrens)
tree.root.labels_transform

### Evaluate in old classes

In [ ]:
Y_hat = tree.inference(old_classes_X)
Y_true = y_train[y_train<num_initial_classes]
print(Y_hat)
np.sum(Y_true==Y_hat)/len(Y_true)

### Evaluate in all classes

In [ ]:
Y_hat = tree.inference(merged_classes_X)
Y_true = y_train[y_train<7]
print(Y_hat)
np.sum(Y_true==Y_hat)/len(Y_true)

### Retraing with all classes

In [ ]:
tree.train(merged_classes_X, merged_classes_Y)

### Re-evaluate in old classes

In [ ]:
Y_hat = tree.inference(old_classes_X)
Y_true = y_train[y_train<num_initial_classes]
print(Y_hat)
np.sum(Y_true==Y_hat)/len(Y_true)

### Re-evaluate in all classes

In [ ]:
Y_hat = tree.inference(merged_classes_X)
Y_true = y_train[y_train<7]
print(Y_hat)
np.sum(Y_true==Y_hat)/len(Y_true)

## Test with CUBS

In [50]:
tree = TreeCNN([(i+1) for i in range(20)], input_shape=(500, 335, 3))
tree.root.net.model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 500, 335, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 506, 341, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 250, 168, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 250, 168, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [7]:
path_base = 'CUB_200_2011/CUB_200_2011/'
path_base = 'CUB_200_2011/' # IN DGX
images_path_map = pd.read_csv(path_base + 'images.txt', sep=' ', header=None)

images_path_map = images_path_map.values
images_path_map[:,0] = images_path_map[:,0] - 1
images_path_map

map_path = {}
for id_, path in images_path_map:
    map_path[id_] = path_base + 'images/' + path
map_path

train_split = pd.read_csv(path_base + 'train_test_split.txt', sep=' ', header=None)
train_mask = train_split.values[:,1]
train_mask = train_mask==1

imageId_label_map = pd.read_csv(path_base + 'image_class_labels.txt', sep=' ', header=None)
imageId_label_map = imageId_label_map.values
imageId_label_map[:, 0 ] = imageId_label_map[:, 0 ] -1

imageId_label_map
map_imageId_label = {}
for id_, cls in imageId_label_map:
    map_imageId_label[id_] = cls


In [8]:
sizes = (335, 500, 3)
map_cv2_img = {}
print(len(map_path.keys()))

for idx in map_path.keys():
    if idx%100==0:
            print(idx)

    img = cv2.imread(map_path[idx])/255.
    img = cv2.resize(img, (sizes[0], sizes[1]))

    map_cv2_img[idx] = img


11788
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700


In [9]:
def create_dataset(num_classes = 20, start_at=1):
    print('\tCreating dataset...')
    idx_used = (imageId_label_map[:,1] >= start_at) * (imageId_label_map[:,1] < (start_at+num_classes))
    
    imgs_id_to_dataset = imageId_label_map[idx_used][:, 0]
    train_mask_used = train_mask[idx_used]
    
    dataset = []
    sizes = (335, 500, 3)

    X = []
    Y = []
    for idx in imgs_id_to_dataset:
        if idx%100==0:
            print('\t\t', idx,'/',len(imgs_id_to_dataset))
        img = map_cv2_img[idx]
        
        X.append(img)
        label = map_imageId_label[idx]
        Y.append(label)
    
    print('\t\tConverting to numpy...')
    X = np.array(X)
    Y = np.array(Y)
    
    print('\t\tShuffle dataset')
    shuffle_idxs = np.array([i for i in range(len(X))])
    np.random.shuffle(shuffle_idxs)
    for idx in range(len(shuffle_idxs)):
        shuffle_idx = shuffle_idxs[idx]
        X_aux = X[shuffle_idx]
        X[shuffle_idx] = X[idx]
        X[idx] = X_aux
        Y_aux = Y[shuffle_idx]
        Y[shuffle_idx] = Y[idx]
        Y[idx] = Y_aux
        
    
#     train_idx = train_mask_used==True
#     test_idx = train_mask_used==False
    print('\t\tSplitting in train and test')
    return X[:int(X.shape[0]*.7)], Y[:int(X.shape[0]*.7)], X[int(X.shape[0]*.7):], Y[int(X.shape[0]*.7):]


In [37]:
# Validating method
x_train, y_train, x_test, y_test = create_dataset(1)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(len(np.unique(y_train)))
print(len(np.unique(y_test)))

	Creating dataset...
		 0 / 60
		Converting to numpy...
		Shuffle dataset
		Splitting in train and test
(42, 500, 335, 3)
(42,)
(18, 500, 335, 3)
(18,)
1
1


In [51]:
accuracy_after_retrain = []
accuracy_before_retrain =[]

In [52]:
for k in range(10):
    print('In iteration: ', (k+1))
    x_train, y_train, x_test, y_test = create_dataset(num_classes=(k+1)*20)
    print('\t * dataset created;')
    # Evaluate before train
    Y_hat = tree.inference(x_test)
    print('\t * Evaluated before train ;')
    Y_true = y_test
    acc = np.sum(Y_true==Y_hat)/len(Y_true)
    accuracy_before_retrain.append(acc)
    
    # Add 20 new classes
    if k != 0:
        print('\t * Adding new classes ;')
        x_train_to_add, y_train_to_add, _, _ = create_dataset(num_classes=20, start_at=((k*20)+1))
        to_insert = [x_train_to_add[y_train_to_add==idx] for idx in np.unique(y_train_to_add)]
        labels = [idx for idx in np.unique(y_train_to_add)]
        tree.addTasks(to_insert, labels)
        
    # train
    print('\t * Trainning;')
    tree.train(x_train, y_train)
    
    # Evaluate after train
    print('\t * Evaluated after train ;')
    Y_hat = tree.inference(x_test)
    Y_true = y_test
    acc = np.sum(Y_true==Y_hat)/len(Y_true)
    accuracy_after_retrain.append(acc)
    
    x_train = None
    y_train = None
    x_test = None
    y_test = None
    Y_true = None
    Y_hat = None

In iteration:  1
	Creating dataset...
		 0 / 1115
		 100 / 1115
		 200 / 1115
		 300 / 1115
		 400 / 1115
		 500 / 1115
		 600 / 1115
		 700 / 1115
		 800 / 1115
		 900 / 1115
		 1000 / 1115
		 1100 / 1115
		Converting to numpy...
		Shuffle dataset
		Splitting in train and test
	 * dataset created;
	 * Evaluated before train ;
	 * Trainning;
1 {1: [1], 2: [2], 3: [3], 4: [4], 5: [5], 6: [6], 7: [7], 8: [8], 9: [9], 10: [10], 11: [11], 12: [12], 13: [13], 14: [14], 15: [15], 16: [16], 17: [17], 18: [18], 19: [19], 20: [20]}
(780, 500, 335, 3) (780, 20)
Train on 702 samples, validate on 78 samples
Epoch 1/100000
702/702 [==============================] - 18s 25ms/sample - loss: 4.0489 - accuracy: 0.0641 - val_loss: 3.0431 - val_accuracy: 0.0385
Epoch 2/100000
702/702 [==============================] - 9s 12ms/sample - loss: 3.3113 - accuracy: 0.0897 - val_loss: 3.0873 - val_accuracy: 0.0256
Epoch 3/100000
702/702 [==============================] - 9s 12ms/sample - loss: 3.1865 - accuracy

		Shuffle dataset
		Splitting in train and test
	 * Trainning;
1 {1: [1], 2: [2], 3: [3], 4: [4], 5: [5], 6: [6], 7: [7], 8: [8], 9: [9], 10: [10], 11: [11], 12: [12], 13: [13], 14: [14], 15: [15], 16: [16], 17: [17], 18: [18], 19: [19], 20: [20], 34: [34], 38: [38], 27: [27], 32: [32], 33: [33], 28: [28], 26: [26], 21: [21], 37: [37], 40: [40], 36: [36], 30: [30], 35: [35], 29: [29], 31: [31], 39: [39], 24: [24], 25: [25], 22: [22], 23: [23], 43: [43], 41: [41], 44: [44], 54: [54], 45: [45], 52: [52], 60: [60], 46: [46], 49: [49], 50: [50], 53: [53], 59: [59], 55: [55], 47: [47], 58: [58], 51: [51], 56: [56], 48: [48], 57: [57], 42: [42]}
(2440, 500, 335, 3) (2440, 60)
Train on 2196 samples, validate on 244 samples
Epoch 1/100000
2196/2196 [==============================] - 35s 16ms/sample - loss: 3.6427 - accuracy: 0.1717 - val_loss: 5.6605 - val_accuracy: 0.0369
Epoch 2/100000
2196/2196 [==============================] - 27s 12ms/sample - loss: 3.4617 - accuracy: 0.1867 - val_loss: 

		Shuffle dataset
		Splitting in train and test
	 * dataset created;
	 * Evaluated before train ;
	 * Adding new classes ;
	Creating dataset...
		 4700 / 1192
		 4800 / 1192
		 4900 / 1192
		 5000 / 1192
		 5100 / 1192
		 5200 / 1192
		 5300 / 1192
		 5400 / 1192
		 5500 / 1192
		 5600 / 1192
		 5700 / 1192
		 5800 / 1192
		Converting to numpy...
		Shuffle dataset
		Splitting in train and test
	 * Trainning;
1 {1: [1], 2: [2], 3: [3], 4: [4], 5: [5], 6: [6], 7: [7], 8: [8], 9: [9], 10: [10], 11: [11], 12: [12], 13: [13], 14: [14], 15: [15], 16: [16], 17: [17], 18: [18], 19: [19], 20: [20], 34: [34], 38: [38], 27: [27], 32: [32], 33: [33], 28: [28], 26: [26], 21: [21], 37: [37], 40: [40], 36: [36], 30: [30], 35: [35], 29: [29], 31: [31], 39: [39], 24: [24], 25: [25], 22: [22], 23: [23], 43: [43], 41: [41], 44: [44], 54: [54], 45: [45], 52: [52], 60: [60], 46: [46], 49: [49], 50: [50], 53: [53], 59: [59], 55: [55], 47: [47], 58: [58], 51: [51], 56: [56], 48: [48], 57: [57], 42: [42], 77:

ResourceExhaustedError:  OOM when allocating tensor with shape[16,512,63,42] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node resnet50/conv3_block4_3_conv/Conv2D (defined at /home/everton/anaconda3/envs/ammd/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1751) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_distributed_function_1359033]

Function call stack:
distributed_function


In [53]:
accuracy_after_retrain

[0.050746268656716415,
 0.033478893740902474,
 0.029636711281070746,
 0.024251069900142655,
 0.02784090909090909]

In [54]:
accuracy_before_retrain

[0.050746268656716415,
 0.036390101892285295,
 0.022944550669216062,
 0.03138373751783167,
 0.0375,
 0.051683262209578]